In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
# !pip3 install http://download.pytorch.org/whl/cu80/torch-0.3.0.post4-cp36-cp36m-linux_x86_64.whl
# !pip3 install torchvision

In [2]:
!apt-get update !apt-get install g++ openjdk-8-jdk python-dev python3-dev
!pip3 install JPype1-py3 
!pip3 install konlpy 
!JAVA_HOME="C:\Program Files\Java\jdk1.8.0_271"


E: The update command takes no arguments
     |████████████████████████████████| 92kB 8.0MB/s 
  Created wheel for JPype1-py3: filename=JPype1_py3-0.5.5.4-cp36-cp36m-linux_x86_64.whl size=2679166 sha256=8a45467dde2b77fe028eb6743b7613069e17746a3101c0f8ced0e2dd644f9299
  Stored in directory: /root/.cache/pip/wheels/52/37/1f/1015d908d12a0e9b239543d031fda0cded9823aa1306939541
Successfully built JPype1-py3
     |████████████████████████████████| 19.4MB 1.3MB/s 
     |████████████████████████████████| 92kB 9.4MB/s 
     |████████████████████████████████| 460kB 57.3MB/s 
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3


In [4]:
import csv
import pandas as pd
import numpy as np
import torch
from konlpy.tag import Komoran
import torch.nn as nn
import torch.nn.functional as F

from collections import defaultdict

# 데이터 파트 안하셔도 되고 노션에서 데이터 다운로드하신 후에 맨아래 공통파트만 하시면 됩니다!
############하이퍼파라미터
#device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 공통파트
# 데이터 전처리
csv.field_size_limit(100000000)
from torchtext import data

tokenizer = Komoran()
#데이터 필드 정의
TEXT = data.Field(sequential=True,
                  use_vocab=True,
                  tokenize=tokenizer.morphs,
                  lower=True,
                  batch_first=True,
                  fix_length=60)

LABEL = data.Field(sequential=False,
                   use_vocab=False,
                   batch_first=False,
                   is_target=True,
                   dtype=torch.float)


from torchtext.data import TabularDataset


train_data, test_data = TabularDataset.splits(path='.', train='/content/drive/MyDrive/Colab Notebooks/악플맞춤법_train.csv', test='/content/drive/MyDrive/Colab Notebooks/악플맞춤법_test.csv', format='csv', fields=[('text', TEXT), ('label', LABEL)], skip_header=True)

print(vars(train_data[0]),vars(train_data[1])) #0번째 1번째 데이터 출력
print(train_data.shape)

TEXT.build_vocab(train_data, min_freq=5, max_size=20000) #최대 2만개, 5번 이상 나오는 단어로 단어사전
#unk=0, pad=1
vocab_size = len(TEXT.vocab)
print(vocab_size)
train_data, val_data = train_data.split(split_ratio=0.8) #train_data와 val_data에 train_data를 8:2로 나눔

train_iter, val_iter, test_iter = data.BucketIterator.splits(
    (train_data, val_data, test_data), batch_size=100,
    shuffle=True, repeat=False, sort=False) #하나의 이터레이터에 batch_size만큼의 묶음으로 저장


{'text': ['유명', 'a', '씨', '의', '딸', '이', '음주', '뺑소니', '를', '하', '고', '도', '회사', '법무', '팀', '이', '고용', '하', 'ㄴ', '로펌', '의', '끈질기', 'ㄴ', '변호', '덕', '에', '집행유예', '를', '이끌', '어', '내', '었', '습니다', '.', '이건', '우리', '사법', '엑사', '에', '쾌거', '이', 'ㅂ니다', '.', '음주', '뺑소니', '재범', '하', '시', 'ㄴ', '재력가', '들', '은', '억울', '하', '게', '옥', '사리', '하지', '마시', '고', '전화', '주', '시', '어요', '.', '1588', '~', '로펌', '로펌', '.', '판사', '들', '을', '하버드', '출신', '용병', '으로', '대체', '하', '아서', '이해관계', '를', '단절', '시키', '어라', '~', '그', '길', '이', '유일한', '선택', '.', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't', '\\', 't'], 'label': '0'} {'text': ['반성', '하', 'ㄴ다고', '집행유예', '이', '라', '…', '진짜', '진짜', '넓', '게', '생각', '하', '아서', '한',

In [5]:
######################CNN 모델
class CNN(nn.Module):
    def __init__(self, vocab_size, embedding_dim, n_filters, filter_sizes, output_dim,
                 dropout, pad_idx):
        super().__init__()

        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=pad_idx)

        self.convs = nn.ModuleList([
            nn.Conv1d(in_channels=embedding_dim,
                      out_channels=n_filters,
                      kernel_size=fs)
            for fs in filter_sizes
        ])

        self.fc = nn.Linear(len(filter_sizes) * n_filters, output_dim)

        self.dropout = nn.Dropout(dropout)

    def forward(self, text):
        # text = [batch size, sent len]

        embedded = self.embedding(text)

        # embedded = [batch size, sent len, emb dim]

        embedded = embedded.permute(0, 2, 1)

        # embedded = [batch size, emb dim, sent len]

        conved = [F.selu(conv(embedded)) for conv in self.convs]

        # conved_n = [batch size, n_filters, sent len - filter_sizes[n] + 1]

        pooled = [F.max_pool1d(conv, conv.shape[2]).squeeze(2) for conv in conved]

        # pooled_n = [batch size, n_filters]

        cat = self.dropout(torch.cat(pooled, dim=1))

        cnn1 =  torch.sigmoid(cat)

        # cat = [batch size, n_filters * len(filter_sizes)]

        return self.fc(cnn1)

In [6]:
def binary_accuracy(preds, y):
    """
    Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
    """

    #round predictions to the closest integer
    rounded_preds = torch.round(torch.sigmoid(preds))
    correct = (rounded_preds == y).float() #convert into float for division
    acc = correct.sum() / len(correct)
    return acc

In [7]:
def accuracy(probs, target):
  predictions = probs.argmax(dim=1)
  corrects = (predictions == target)
  accuracy = corrects.sum().float() / float(target.size(0))
  return accuracy


In [8]:
import time

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

In [9]:
import torch 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [10]:
def evaluate(model, iterator, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.eval()

    with torch.no_grad():
        for batch in iterator:
            predictions = model(batch.text).squeeze(1)

            loss = criterion(predictions, batch.label)

            acc = binary_accuracy(predictions, batch.label)

            epoch_loss += loss.item()
            epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [11]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_acc = 0

    model.train()

    for batch in iterator:
        optimizer.zero_grad()

        predictions = model(batch.text).squeeze(1)

        loss = criterion(predictions, batch.label)

        acc = binary_accuracy(predictions, batch.label)

        loss.backward()

        optimizer.step()

        epoch_loss += loss.item()
        epoch_acc += acc.item()

    return epoch_loss / len(iterator), epoch_acc / len(iterator)

In [12]:
import torch
import os
INPUT_DIM = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_FILTERS = 50
FILTER_SIZES = [2,3,4,5,6,7,8,9]
#FILTER_SIZES = [2,3,4]
OUTPUT_DIM = 1
DROPOUT = 0.8
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

###
# EMBEDDING_DIM = 100
# N_FILTERS = 100
# FILTER_SIZES = [3,4,5]
# OUTPUT_DIM = 1

model = CNN(INPUT_DIM, EMBEDDING_DIM, N_FILTERS, FILTER_SIZES, OUTPUT_DIM, DROPOUT, PAD_IDX)


In [13]:
model.convs
model

CNN(
  (embedding): Embedding(13481, 100, padding_idx=1)
  (convs): ModuleList(
    (0): Conv1d(100, 50, kernel_size=(2,), stride=(1,))
    (1): Conv1d(100, 50, kernel_size=(3,), stride=(1,))
    (2): Conv1d(100, 50, kernel_size=(4,), stride=(1,))
    (3): Conv1d(100, 50, kernel_size=(5,), stride=(1,))
    (4): Conv1d(100, 50, kernel_size=(6,), stride=(1,))
    (5): Conv1d(100, 50, kernel_size=(7,), stride=(1,))
    (6): Conv1d(100, 50, kernel_size=(8,), stride=(1,))
    (7): Conv1d(100, 50, kernel_size=(9,), stride=(1,))
  )
  (fc): Linear(in_features=400, out_features=1, bias=True)
  (dropout): Dropout(p=0.8, inplace=False)
)

In [ ]:
if __name__ == "__main__":
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(device)
    model_type = "CNN"  # or: "token"

    data_type = "morph"  # or: "token"

    criterion = nn.BCEWithLogitsLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.003) #경사하강법 Adam
    N_EPOCHS = 15

    best_valid_loss = float('inf')

    for epoch in range(N_EPOCHS):

        start_time = time.time()
        train_loss, train_acc = train(model, train_iter, optimizer, criterion)
        valid_loss, valid_acc = evaluate(model, val_iter, criterion)

        end_time = time.time()

        epoch_mins, epoch_secs = epoch_time(start_time, end_time)

        if valid_loss < best_valid_loss:
            best_valid_loss = valid_loss
            torch.save(model.state_dict(), 'tut4-model.pt')

        print(f'Epoch: {epoch + 1:02} | Epoch Time: {epoch_mins}m {epoch_secs}s')
        print(f'\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc * 100:.2f}%')
        print(f'\t Val. Loss: {valid_loss:.3f} |  Val. Acc: {valid_acc * 100:.2f}%')

model.load_state_dict(torch.load('tut4-model.pt'))

test_loss, test_acc = evaluate(model, test_iter, criterion)

print(f'Test Loss: {test_loss:.3f} | Test Acc: {test_acc*100:.2f}%')

cuda
Epoch: 01 | Epoch Time: 2m 15s
	Train Loss: 0.562 | Train Acc: 79.20%
	 Val. Loss: 1.915 |  Val. Acc: 79.22%


In [ ]:
from torchtext import data

tokenizer = Komoran()
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
def predict_sentiment(model, sentence, min_len = 60):
    model.eval().to(device)
    tokenized = [tok for tok in tokenizer.morphs(sentence)]
    print(tokenized)
    if len(tokenized) < min_len:
        tokenized += ['<pad>'] * (min_len - len(tokenized))
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(0)
    length_tensor = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor))
    return prediction.item()

In [ ]:
predict_sentiment(model, "뀨")

In [ ]:
# !nvcc --version

In [ ]:
# pip install torch==1.7.0+cu101 torchvision==0.8.1+cu101 torchaudio===0.7.0 -f https://download.pytorch.org/whl/torch_stable.html